In [2]:
import pandas as pd
from pathlib import Path

In [3]:
csv_paths = []
combined_path = Path("../data/qna/combined")
for file in combined_path.iterdir():
    csv_paths.append(Path(file))
    

In [4]:
csv_paths

[PosixPath('../data/qna/combined/mhqa-b.csv'),
 PosixPath('../data/qna/combined/bioasq.csv'),
 PosixPath('../data/qna/combined/medqa.csv'),
 PosixPath('../data/qna/combined/mhqa.csv'),
 PosixPath('../data/qna/combined/medquad.csv'),
 PosixPath('../data/qna/combined/medmcqa.csv'),
 PosixPath('../data/qna/combined/preprocessed'),
 PosixPath('../data/qna/combined/pubmed.csv')]

In [7]:
idx = 4
csv_path = csv_paths[idx]
print(csv_path)
df = pd.read_csv(csv_path)
df.info()

../data/qna/combined/medquad.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2995 entries, 0 to 2994
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   document_id          2995 non-null   int64 
 1   document_source      2995 non-null   object
 2   document_url         2995 non-null   object
 3   category             2995 non-null   object
 4   umls_cui             2995 non-null   object
 5   umls_semantic_types  2995 non-null   object
 6   umls_semantic_group  2995 non-null   object
 7   synonyms             2995 non-null   object
 8   question_id          2995 non-null   object
 9   question_focus       2995 non-null   object
 10  question_type        2995 non-null   object
 11  Question             2995 non-null   object
 12  answer               2995 non-null   object
dtypes: int64(1), object(12)
memory usage: 304.3+ KB


In [8]:
df.head()

,document_id,document_source,document_url,category,umls_cui,umls_semantic_types,umls_semantic_group,synonyms,question_id,question_focus,question_type,Question,answer
0,985,GHR,https://ghr.nlm.nih.gov/condition/tourette-syn...,\N,C0040517,T047,Disorders,Chronic Motor and Vocal Tic Disorder|Gilles de...,0000985-1,Tourette syndrome,information,What is (are) Tourette syndrome ?,Tourette syndrome is a complex disorder charac...
1,985,GHR,https://ghr.nlm.nih.gov/condition/tourette-syn...,\N,C0040517,T047,Disorders,Chronic Motor and Vocal Tic Disorder|Gilles de...,0000985-2,Tourette syndrome,frequency,How many people are affected by Tourette syndr...,Although the exact incidence of Tourette syndr...
2,985,GHR,https://ghr.nlm.nih.gov/condition/tourette-syn...,\N,C0040517,T047,Disorders,Chronic Motor and Vocal Tic Disorder|Gilles de...,0000985-3,Tourette syndrome,genetic changes,What are the genetic changes related to Touret...,A variety of genetic and environmental factors...
3,985,GHR,https://ghr.nlm.nih.gov/condition/tourette-syn...,\N,C0040517,T047,Disorders,Chronic Motor and Vocal Tic Disorder|Gilles de...,0000985-4,Tourette syndrome,inheritance,Is Tourette syndrome inherited ?,The inheritance pattern of Tourette syndrome i...
4,985,GHR,https://ghr.nlm.nih.gov/condition/tourette-syn...,\N,C0040517,T047,Disorders,Chronic Motor and Vocal Tic Disorder|Gilles de...,0000985-5,Tourette syndrome,treatment,What are the treatments for Tourette syndrome ?,These resources address the diagnosis or manag...


In [9]:
df.rename(columns={'Question': "question", "question_id": "id"}, inplace=True, errors='ignore')
print("Columns: ",df.columns.tolist())

Columns:  ['document_id', 'document_source', 'document_url', 'category', 'umls_cui', 'umls_semantic_types', 'umls_semantic_group', 'synonyms', 'id', 'question_focus', 'question_type', 'question', 'answer']


In [10]:
cols = set(df.columns.tolist()) - set(['id', "question", "answer"])
df.drop(columns=cols, inplace=True, errors='ignore')
print("Remaining Columns: ", df.columns.tolist())

Remaining Columns:  ['id', 'question', 'answer']


In [11]:
df.head()

,id,question,answer
0,0000985-1,What is (are) Tourette syndrome ?,Tourette syndrome is a complex disorder charac...
1,0000985-2,How many people are affected by Tourette syndr...,Although the exact incidence of Tourette syndr...
2,0000985-3,What are the genetic changes related to Touret...,A variety of genetic and environmental factors...
3,0000985-4,Is Tourette syndrome inherited ?,The inheritance pattern of Tourette syndrome i...
4,0000985-5,What are the treatments for Tourette syndrome ?,These resources address the diagnosis or manag...


In [110]:
df['cop'] = df['cop'].apply(lambda x: x+1)
df['cop'].unique()

array([3, 4, 2, 1, 0])

In [111]:
invalid_cop_count = df[~df['cop'].between(1, 4, inclusive='both')].shape[0]
df = df[df['cop'].between(1, 4, inclusive='both')]
print(f"Removed {invalid_cop_count} rows with 'cop' values outside the range [1,4]")

Removed 215 rows with 'cop' values outside the range [1,4]


In [112]:
def func(row):
    col = f"option{row['cop']}"
    return row[col]

df['answer'] = df.apply(func, axis=1)
df.head()

,id,question,option1,option2,option3,option4,cop,answer
0,cbc8a1b1-532c-490e-a17d-9168083fabae,Mark the true statement regarding use of Lithi...,Monotherapy for acute episodes,Monitoring of serum concentration is not usefu...,Can result in leucocytosis,Can result in hypehyroidism on long term use,3,Can result in leucocytosis
1,01b9f068-4db2-45e5-94b8-19d4f7d290f7,A 14-year-old girl presents with quadriparesis...,Emery-Dreifuss muscular dystrophy,Becker muscular dystrophy,Limb-girdle dystrophy,Scapulofaciohumeral dystrophy,4,Scapulofaciohumeral dystrophy
2,fec423fb-be98-40a8-a9f3-477f6e7555f4,Schizophrenia is treated by,Anti depressants,Anti psychotics,Anti epileptics,Mood stabilizers,2,Anti psychotics
3,090f4f67-eab2-4ffc-a9d0-d03da739bf9f,A 19 year old female presents with pain in the...,Atropine injection,Levothyroxine administration,Aspirin,Increase iodine intake in food,3,Aspirin
4,b37eb019-4f8c-4131-8ff9-c2413a5b227e,"Headache, apathy and deteriorating level of co...",Chronic subdral haematoma.,Pontine hemorrhage,Continuing cerebral.,Depressed skull fracture.,1,Chronic subdral haematoma.


In [12]:
prev_len = len(df)
df.drop_duplicates(['question','answer'], inplace=True)
print("# Duplicates Found: ", prev_len - len(df))

# Duplicates Found:  58


In [13]:
prev_len = len(df)
df.dropna(inplace=True)
print("# Null Values: ", prev_len - len(df))


# Null Values:  0


In [17]:
df.head()

,id,question,answer
0,0000985-1,What is (are) Tourette syndrome ?,Tourette syndrome is a complex disorder charac...
1,0000985-2,How many people are affected by Tourette syndr...,Although the exact incidence of Tourette syndr...
2,0000985-3,What are the genetic changes related to Touret...,A variety of genetic and environmental factors...
3,0000985-4,Is Tourette syndrome inherited ?,The inheritance pattern of Tourette syndrome i...
4,0000985-5,What are the treatments for Tourette syndrome ?,These resources address the diagnosis or manag...


In [15]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(
    df, 
    test_size=0.2, 
    random_state=42
)

print(f"Original dataframe shape: {df.shape}")
print(f"Train dataframe shape: {train_df.shape}")
print(f"Test dataframe shape: {test_df.shape}")

Original dataframe shape: (2937, 3)
Train dataframe shape: (2349, 3)
Test dataframe shape: (588, 3)


In [128]:
parent_path = csv_path.parent / "preprocessed"
parent_path.mkdir(exist_ok=True)
train_save_path = parent_path / (csv_path.stem + "_train.csv")
test_save_path = parent_path / (csv_path.stem + "_test.csv")

train_df.to_csv(train_save_path, index=False)
test_df.to_csv(test_save_path, index=False)

print("Saved train_df to ", train_save_path)
print("Saved test_df to ", test_save_path)

Saved train_df to  ../data/qna/combined/preprocessed/pubmed_train.csv
Saved test_df to  ../data/qna/combined/preprocessed/pubmed_test.csv
